In [1]:
from transformers import AutoProcessor, GroundingDinoForObjectDetection
from PIL import Image
import requests
import torch

device = 'cuda'

processor = AutoProcessor.from_pretrained("IDEA-Research/grounding-dino-tiny")
model = GroundingDinoForObjectDetection.from_pretrained("IDEA-Research/grounding-dino-tiny")
model = model.eval().to(device)

In [2]:
torch.cuda.empty_cache()

image = Image.open('../images/7.png')
text = "dustbin"
image = image.convert('RGB')

inputs = processor(images=image, text=text, return_tensors="pt").to(device)

outputs = model(**inputs)

In [4]:
# convert outputs (bounding boxes and class logits) to COCO API
target_sizes = torch.tensor([image.size[::-1]])
results = processor.image_processor.post_process_object_detection(
    outputs, threshold=0.1, target_sizes=target_sizes
)[0]
for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
    box = [round(i, 1) for i in box.tolist()]
    print(f"Detected {label.item()} with confidence " f"{round(score.item(), 2)} at location {box}")

Detected 2 with confidence 0.12 at location [186.2, 186.6, 297.0, 307.9]


In [5]:
# visualize the results
import supervision as sv

image = sv.utils.draw_bounding_boxes(image, results["boxes"], results["labels"], results["scores"])
image.show()

AttributeError: module 'supervision.utils' has no attribute 'draw_bounding_boxes'